# AlexNet à partir de zéro
## Introduction
AlexNet est un réseau neuronal convolutif profond, qui a été initialement développé par Alex Krizhevsky et ses collègues en 2012. Il a été conçu pour classer les images pour le concours ImageNet LSVRC-2010 où il a obtenu des résultats de pointe.
Ici, je vais résumer les principaux points à retenir sur le réseau AlexNet. Tout d’abord, il fonctionnait avec des images à 3 canaux de taille (224x224x3). Il utilisait le regroupement maximal ainsi que les activations ReLU lors du sous-échantillonnage. Les noyaux utilisés pour les convolutions étaient soit 11x11, 5x5 ou 3x3 tandis que les noyaux utilisés pour le regroupement maximal avaient une taille de 3x3. Il a classé les images en 1000 classes. Il utilisait également plusieurs GPU.
## Chargement des données
### Jeu de données
Commençons par charger puis prétraiter les données. Pour nos besoins, nous utiliserons le CIFAR10 ensemble de données. L’ensemble de données se compose de 60000 images couleur 32x32 dans 10 classes, avec 6000 images par classe. Il y a 50000 images d’entraînement et 10000 images de test.

Voici les classes de l’ensemble de données, ainsi que 10 exemples d’images aléatoires de chacune :

<img src="classes.png" alt="classes" width="500px">

Les classes s’excluent mutuellement. Il n’y a pas de chevauchement entre les automobiles et les camions. « Automobile » comprend les berlines, les SUV et les choses de ce genre. Le terme « camion » ne comprend que les gros camions. Ni l’un ni l’autre n’inclut les camionnettes.


## Importation des bibliothèques
Commençons par importer les bibliothèques requises et définir une variable `device`, afin que le notebook sache utiliser un GPU pour entraîner le modèle s’il est disponible.

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler


# Configuration de l'appareil
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## **Chargement du jeu de données**
En utilisant `torchvision` (une bibliothèque d’assistance pour les tâches de vision par ordinateur), nous chargerons notre ensemble de données. Cette méthode a des fonctions d’assistance qui rendent le prétraitement assez facile et direct. Définissons les fonctions `get_train_valid_loader` et `get_test_loader`, puis appelons-les pour charger et traiter notre donnée CIFAR-10 :

In [2]:
def get_train_valid_loader(data_dir,
                           batch_size,
                           augment,
                           random_seed,
                           valid_size=0.1,
                           shuffle=True):
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    valid_transform = transforms.Compose([
            transforms.Resize((227,227)),
            transforms.ToTensor(),
            normalize,
    ])
    if augment:
        train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])
    else:
        train_transform = transforms.Compose([
            transforms.Resize((227,227)),
            transforms.ToTensor(),
            normalize,
        ])

    # load the dataset
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=train_transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=valid_transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler)
 
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)


def get_test_loader(data_dir,
                    batch_size,
                    shuffle=True):
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

    # define transform
    transform = transforms.Compose([
        transforms.Resize((227,227)),
        transforms.ToTensor(),
        normalize,
    ])

    dataset = datasets.CIFAR10(
        root=data_dir, train=False,
        download=True, transform=transform,
    )

    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle
    )

    return data_loader


# CIFAR10 dataset 
train_loader, valid_loader = get_train_valid_loader(data_dir = './data', batch_size = 64, augment = False, random_seed = 1)

test_loader = get_test_loader(data_dir = './data',
                              batch_size = 64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


Décomposons le code en morceau :
- Nous définissons deux fonctions `get_train_valid_loader` et `get_test_loader` pour charger respectivement les ensembles de formation/validation et de test.
- Nous commençons par définir la variable `normalize` avec la moyenne et les écarts-types de chacun des canaux (rouge, vert et bleu) dans l’ensemble de données. Ceux-ci peuvent être calculés manuellement, mais sont également disponibles en ligne puisque CIFAR-10 est très populaire.
- Pour notre jeu de données d’entraînement, nous ajoutons l’option d’augmenter également le jeu de données pour un entraînement plus robuste et augmenter le nombre d’images. Remarque : l’augmentation n’est appliquée qu’au sous-ensemble d’apprentissage et non aux sous-ensembles de validation et de test, car ils ne sont utilisés qu’à des fins d’évaluation.
- Nous divisons le jeu de données d’entraînement en ensembles d’entraînement et de validation (ratio 90:10) et le sous-ensemble aléatoirement à partir de l’ensemble d’entraînement.
- Nous spécifions la taille du lot et mélangeons le jeu de données lors du chargement, de sorte que chaque lot présente une certaine variation dans les types d’étiquettes qu’il possède. Cela augmentera l’efficacité de notre modèle éventuel.
- Enfin, nous utilisons des chargeurs de données. Cela peut ne pas affecter les performances dans le cas d’un petit jeu de données comme CIFAR10, mais cela peut vraiment entraver les performances dans le cas de grands jeux de données et est généralement considéré comme une bonne pratique. Les chargeurs de données nous permettent d’itérer sur les données par lots, et les données sont chargées pendant l’itération et non toutes en même temps dans votre RAM

## AlexNet à partir de zéro
Commençons par le code:

In [3]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

Voyons comment fonctionne le code ci-dessus :

- La première étape pour définir un réseau neuronal (qu’il s’agisse d’un CNN ou non) dans PyTorch est de définir une classe qui hérite `nn.Module` car elle contient de nombreuses méthodes que nous devrons utiliser.
- Il y a deux étapes principales après cela. La première consiste à initialiser les couches que nous allons utiliser dans notre CNN à l’intérieur de `__init__`, et l’autre consiste à définir l’ordre dans lequel ces couches traiteront l’image. Ceci est défini à l’intérieur de la fonction `forward`.
- Pour l’architecture elle-même, nous définissons d’abord les couches convolutives à l’aide de la fonction `nn.Conv2D` avec la taille de noyau appropriée et les canaux d’entrée/sortie. Nous appliquons également la fonction de mise en commun maximale `nn.MaxPool2D`. La bonne chose à propos de PyTorch est que nous pouvons combiner la couche convolutive, la fonction d’activation et le pool maximum en une seule couche (ils seront appliqués séparément, mais cela aide à l’organisation) en utilisant la fonction `nn.Sequential`
- Ensuite, nous définissons les couches entièrement connectées en utilisant linear (`nn.Linear`) et dropout (`nn.Dropout`) ainsi que la fonction d’activation ReLu (`nn.ReLU`) et les combinons avec la fonction `nn.Sequential`.
- Enfin, notre dernière couche produit 10 neurones qui sont nos prédictions finales pour les 10 classes d’objets.

## **Définition des hyperparamètres**
Avant l’entraînement, nous devons définir certains hyperparamètres, tels que la fonction de perte et l’optimiseur à utiliser, ainsi que la taille du lot, le taux d’apprentissage et le nombre d’époques.

In [4]:
num_classes = 10
num_epochs = 20
batch_size = 64
learning_rate = 0.005

model = AlexNet(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(train_loader)

Nous commençons par définir des hyperparamètres simples (époques, taille de lot et taux d’apprentissage) et initialiser notre modèle en utilisant le nombre de classes comme argument, qui dans ce cas est de 10 avec le transfert du modèle vers le périphérique approprié (CPU ou GPU). Ensuite, nous définissons notre fonction de coût comme perte d’entropie croisée et l’optimiseur comme Adam. Il y a beaucoup de choix pour ceux-ci, mais ceux-ci ont tendance à donner de bons résultats avec le modèle et les données données. Enfin, nous définissons `total_step` pour mieux suivre les étapes lors de l’entraînement

## Entrainement
Nous sommes prêts à entraîner notre modèle à ce stade :

In [5]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))

Epoch [1/20], Step [704/704], Loss: 0.9498
Accuracy of the network on the 5000 validation images: 57.52 %
Epoch [2/20], Step [704/704], Loss: 0.7102
Accuracy of the network on the 5000 validation images: 68.3 %
Epoch [3/20], Step [704/704], Loss: 0.4972
Accuracy of the network on the 5000 validation images: 69.76 %
Epoch [4/20], Step [704/704], Loss: 0.9376
Accuracy of the network on the 5000 validation images: 73.22 %


KeyboardInterrupt: 

Voyons ce que fait le code :

- Nous commençons par itérer sur le nombre d’époques, puis sur les lots dans nos données d’entraînement
- Nous convertissons les images et les étiquettes en fonction de l’appareil que nous utilisons, c’est-à-dire GPU ou CPU
- Dans le forward pass, nous faisons des prédictions à l’aide de notre modèle et calculons la perte en fonction de ces prédictions et de nos étiquettes réelles
- Ensuite, nous effectuons la passe arrière où nous mettons à jour nos poids pour améliorer notre modèle
- Nous mettons ensuite les dégradés à zéro avant chaque mise à jour à l’aide de la fonction `optimizer.zero_grad()`
- Ensuite, nous calculons les nouveaux gradients à l’aide de la fonction `loss.backward()`
- Et enfin, nous mettons à jour les poids avec la fonction `optimizer.step()`
- De plus, à la fin de chaque époque, nous utilisons également notre jeu de validation pour calculer la précision du modèle. Dans ce cas, nous n’avons pas besoin de gradients, nous utilisons `with torch.no_grad()` donc pour une évaluation plus rapide


Nous pouvons voir la sortie comme suit :

<img src="training_loss_and_Validation_Accuracy.png" alt="Perte d’entraînement et précision de la validation" width="500px">

Comme nous pouvons le voir, la perte diminue à chaque époque, ce qui montre que notre modèle est effectivement en train d’apprendre. Notez que cette perte se trouve sur l’ensemble d’apprentissage, et si la perte est beaucoup trop faible, cela peut indiquer un surapprentissage. C’est pourquoi nous utilisons également le kit de validation. La précision semble augmenter sur l’ensemble de validation, ce qui indique qu’il y a peu de chances de surapprentissage. Testons maintenant notre modèle pour voir comment il fonctionne.

## Test
Maintenant, voyons comment notre modèle se comporte sur des données invisibles :

In [6]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))

Accuracy of the network on the 10000 test images: 74.37 %


Notez que le code est exactement le même que pour nos besoins de validation.

En utilisant le modèle et en nous entraînant pour seulement 6 époques, nous semblons obtenir une précision d’environ 78,8 % sur l’ensemble de validation, ce qui semble suffisant.<br><br>
<img src='testing-Accuracy.png' alt='Précision des tests' width='500px'>

## Conclusion
Concluons maintenant ce que nous avons fait :

- Nous avons commencé par comprendre l’architecture et les différents types de couches du modèle AlexNet
- Ensuite, nous avons chargé et prétraité le jeu de données CIFAR10 à l’aide de `torchvision`
- Ensuite, nous avions l’habitude de construire notre modèle AlexNet à partir de zéro `PyTorch`
- Enfin, nous avons entraîné et testé notre modèle sur le jeu de données CIFAR10, et le modèle semblait bien fonctionner sur le jeu de données de test avec un entraînement minimal (6 époques)